In [1]:

from allennlp.training import GradientDescentTrainer
from allennlp.data.dataset_readers import DatasetReader
from allennlp.data import Instance
from allennlp.models.archival import load_archive
from allennlp.data import DatasetReader, Instance, Field
from allennlp.data.fields import TextField, SpanField, ListField, SequenceLabelField, MetadataField
from allennlp.data.token_indexers import TokenIndexer, PretrainedTransformerMismatchedIndexer
from allennlp.data.tokenizers import Tokenizer, SpacyTokenizer
from allennlp.data.dataset_readers.dataset_utils import enumerate_spans
from typing import Iterator, List, Dict, Any, Optional
import json
# from allennlp_models.coref import CorefPredictor
import allennlp_models.coref #MUST HAVE THIS LINE TO LOAD COREF 
import torch
# Check for CUDA
if torch.cuda.is_available():
    print("CUDA available, predictions will be faster.")
else:
    print("CUDA not available, predictions may be slower.")

CUDA available, predictions will be faster.


In [2]:

# Load the pre-trained archive
archive = load_archive("spanbert_local/coref-spanbert-large-2021.03.10.tar.gz")
model = archive.model
print(archive)
# Make sure to move the model to the correct device
model = model.cuda(0)  # Replace with model.cpu() if you don't want to use the GPU

error loading _jsonnet (this is expected on Windows), treating C:\Users\pc-bae-2\AppData\Local\Temp\tmpip92bhrj\config.json as plain json
Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-large-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Archive(model=CoreferenceResolver(
  (_text_field_embedder): BasicTextFieldEmbedder(
    (token_embedder_tokens): PretrainedTransformerMismatchedEmbedder(
      (_matched_embedder): PretrainedTransformerEmbedder(
        (transformer_model): BertModel(
          (embeddings): BertEmbeddings(
            (word_embeddings): Embedding(28996, 1024, padding_idx=0)
            (position_embeddings): Embedding(512, 1024)
            (token_type_embeddings): Embedding(2, 1024)
            (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (encoder): BertEncoder(
            (layer): ModuleList(
              (0-23): 24 x BertLayer(
                (attention): BertAttention(
                  (self): BertSelfAttention(
                    (query): Linear(in_features=1024, out_features=1024, bias=True)
                    (key): Linear(in_features=1024, out_features=1024, bias=True)
                    (

In [3]:
print(archive.config["dataset_reader"])

train_path = "gap-development.jsonl"  
val_path = "gap-validation.jsonl" 
# Load your training and validation data
reader = DatasetReader.from_params(archive.config["dataset_reader"])

dataset_reader.Params({'type': 'coref', 'max_sentences': 110, 'max_span_width': 30, 'token_indexers': {'tokens': {'type': 'pretrained_transformer_mismatched', 'max_length': 512, 'model_name': 'SpanBERT/spanbert-large-cased'}}})


In [6]:
from allennlp.data import DatasetReader, Instance, Field
from allennlp.data.fields import TextField, SpanField, ListField
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer, PretrainedTransformerMismatchedIndexer
from allennlp.data.tokenizers import Tokenizer, SpacyTokenizer
from typing import Iterator, List, Dict, Any
import json
from allennlp.data.dataset_readers.dataset_utils import enumerate_spans

class CustomCorefReader(DatasetReader):
    def __init__(self,
                 tokenizer: Tokenizer = None,
                 token_indexers: Dict[str, TokenIndexer] = None,
                 max_span_width: int = 30,
                 max_sentences : int = 110,
                 **kwargs: Any):
        super().__init__(**kwargs)
        
        self.max_sentences = max_sentences
        print(f"Received max_sentences: {self.max_sentences}")
        self.tokenizer = tokenizer or SpacyTokenizer()
        self.token_indexers = token_indexers or {"tokens": PretrainedTransformerMismatchedIndexer(model_name="SpanBERT/spanbert-large-cased", max_length=512)}
        self._max_span_width = max_span_width

    def _read(self, file_path: str) -> Iterator[Instance]:
        with open(file_path, 'r') as f:
            for line in f:
                example = json.loads(line)
                text = example['text']
                clusters = example.get('clusters', None)  # clusters could be None during inference
                
                if clusters is not None:
                    clusters = [tuple(map(tuple, cluster)) for cluster in clusters]


                tokens_with_offsets = self.tokenizer.tokenize(text)
                token_offsets = [(token.idx, token.idx + len(token.text)) for token in tokens_with_offsets]

                text_field = TextField(tokens_with_offsets, self.token_indexers)

                # Generate all possible spans up to max_span_width using enumerate_spans
                possible_spans = enumerate_spans(tokens_with_offsets, max_span_width=self._max_span_width)
                span_fields: List[Field] = [SpanField(start, end, text_field) for start, end in possible_spans]

                span_labels = []
                cluster_dict = {}
                if clusters is not None:
                    for cluster_id, cluster in enumerate(clusters):
                        for char_start, char_end in cluster:
                            try:
                                 token_start = next(i for i, (t_start, t_end) in enumerate(token_offsets) if t_start <= char_start < t_end)
                            except StopIteration:
                                    print(f"StopIteration for token_start: char_start={char_start}, token_offsets={token_offsets}")
                                    raise

                            try:
                                    token_end = next(i for i, (t_start, t_end) in enumerate(token_offsets) if t_start < char_end <= t_end)
                            except StopIteration as e:
                                    print(f"StopIteration for token_end: char_end={char_end}, token_offsets={token_offsets}")
                                    # Assuming `text` contains the actual text content.
                                    print(f"Text around char_end: {text[char_end-10:char_end+10]}")
    	                            
                                    continue
                            cluster_dict[(token_start, token_end)] = cluster_id

                    for start, end in possible_spans:
                        span_labels.append(cluster_dict.get((start, end), -1))
                    fields: Dict[str, Field] = {'text': text_field, 'spans': ListField(span_fields), 'span_labels': SequenceLabelField(span_labels, ListField(span_fields))}
                else:
                    fields: Dict[str, Field] = {'text': text_field, 'spans': ListField(span_fields)}

                # Metadata
                metadata = {'original_text': text, 'token_offsets': token_offsets}
                if clusters is not None:
                    metadata['clusters'] = clusters
                fields['metadata'] = MetadataField(metadata)
                yield Instance(fields)

reader = CustomCorefReader()


Received max_sentences: 110


f:\UvA 2023\DL 4 NLP\Project\DLNLP-pronoun-resolution\DLNLP-pronoun-resolution\spanbert_env\lib\site-packages\torch\__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\tensor\python_tensor.cpp:453.)
  _C._set_default_tensor_type(t)


In [7]:
# Read the training and validation data
train_data = list(reader.read(train_path))

validation_data = list(reader.read(val_path))
print(len(train_data))
# Build a vocabulary from the training data
# vocab = Vocabulary.from_instances(train_data)
vocab = model.vocab
# Create data loaders
from allennlp.data.data_loaders import MultiProcessDataLoader
from allennlp.data.samplers import BucketBatchSampler

train_sampler = BucketBatchSampler(batch_size=3, sorting_keys=["text"])

train_data_loader = MultiProcessDataLoader(reader=reader, data_path=train_path, batch_sampler=train_sampler)
validation_data_loader = MultiProcessDataLoader(reader=reader, data_path=val_path, batch_size=3)

# Index the data loaders with the vocabulary
train_data_loader.index_with(vocab)
validation_data_loader.index_with(vocab)

# Make sure your model also uses the same vocabulary
# model.vocab = vocab

# Debugging: Print the number of instances in the data
print("Number of instances in train_data:", len(train_data))
print("Number of instances in validation_data:", len(validation_data))


StopIteration for token_end: char_end=323, token_offsets=[(0, 2), (3, 12), (13, 15), (16, 24), (25, 31), (32, 34), (35, 42), (43, 48), (48, 49), (50, 51), (51, 52), (52, 55), (56, 61), (62, 67), (68, 70), (71, 73), (74, 77), (78, 85), (86, 92), (93, 97), (98, 104), (105, 107), (108, 114), (114, 116), (117, 127), (128, 134), (135, 143), (144, 146), (147, 150), (151, 157), (158, 166), (167, 169), (170, 179), (180, 188), (189, 191), (192, 195), (196, 201), (202, 204), (205, 206), (207, 211), (212, 216), (216, 217), (218, 224), (225, 235), (236, 243), (244, 251), (252, 254), (255, 263), (264, 269), (269, 270), (271, 279), (280, 283), (284, 295), (295, 296), (297, 301), (302, 304), (305, 308), (309, 316), (316, 317), (318, 322), (323, 324), (325, 329), (330, 333), (334, 343), (344, 347), (348, 361), (362, 366), (367, 370), (371, 380), (381, 385), (386, 388), (389, 400), (401, 407), (407, 409), (410, 414), (415, 417), (418, 420), (421, 426), (427, 430), (431, 437), (438, 440), (441, 444), (4

loading instances: 0it [00:00, ?it/s]

StopIteration for token_end: char_end=323, token_offsets=[(0, 2), (3, 12), (13, 15), (16, 24), (25, 31), (32, 34), (35, 42), (43, 48), (48, 49), (50, 51), (51, 52), (52, 55), (56, 61), (62, 67), (68, 70), (71, 73), (74, 77), (78, 85), (86, 92), (93, 97), (98, 104), (105, 107), (108, 114), (114, 116), (117, 127), (128, 134), (135, 143), (144, 146), (147, 150), (151, 157), (158, 166), (167, 169), (170, 179), (180, 188), (189, 191), (192, 195), (196, 201), (202, 204), (205, 206), (207, 211), (212, 216), (216, 217), (218, 224), (225, 235), (236, 243), (244, 251), (252, 254), (255, 263), (264, 269), (269, 270), (271, 279), (280, 283), (284, 295), (295, 296), (297, 301), (302, 304), (305, 308), (309, 316), (316, 317), (318, 322), (323, 324), (325, 329), (330, 333), (334, 343), (344, 347), (348, 361), (362, 366), (367, 370), (371, 380), (381, 385), (386, 388), (389, 400), (401, 407), (407, 409), (410, 414), (415, 417), (418, 420), (421, 426), (427, 430), (431, 437), (438, 440), (441, 444), (4

loading instances: 0it [00:00, ?it/s]

StopIteration for token_end: char_end=285, token_offsets=[(0, 8), (9, 12), (13, 14), (15, 28), (29, 35), (36, 38), (39, 44), (45, 51), (52, 55), (56, 58), (59, 63), (64, 69), (70, 73), (74, 77), (78, 82), (83, 85), (86, 89), (90, 93), (94, 101), (101, 102), (103, 106), (107, 111), (112, 116), (116, 118), (119, 124), (125, 133), (134, 141), (141, 142), (143, 145), (146, 153), (154, 160), (161, 163), (164, 173), (174, 176), (177, 180), (181, 185), (185, 186), (187, 192), (193, 202), (203, 207), (208, 210), (211, 216), (216, 217), (218, 225), (226, 235), (235, 236), (237, 239), (240, 243), (244, 247), (248, 257), (258, 260), (261, 266), (266, 267), (268, 273), (274, 279), (280, 284), (285, 287), (288, 295), (296, 298), (299, 304), (305, 313), (313, 314), (315, 319), (320, 323), (324, 335), (336, 338), (339, 347), (348, 350), (351, 357), (358, 364), (364, 365), (366, 379), (380, 382), (383, 390), (391, 394), (395, 397), (398, 404), (404, 405), (406, 409), (410, 420), (421, 423), (424, 429)

In [8]:
# Print first instance from the training data
print("First instance in train_data:")
print(train_data[0])

# Or, to see it in a more readable format
print("Pretty-printing first instance in train_data:")
for field_name, field in train_data[0].fields.items():
    print(f"{field_name}: {field}")

# Print first instance from the validation data
print("\nFirst instance in validation_data:")
print(validation_data[0])

# Or, to see it in a more readable format
print("Pretty-printing first instance in validation_data:")
for field_name, field in validation_data[0].fields.items():
    print(f"{field_name}: {field}")


First instance in train_data:
Instance with fields:
 	 text: TextField of length 86 with text: 
 		[Zoe, Telford, --, played, the, police, officer, girlfriend, of, Simon, ,, Maggie, ., Dumped, by,
		Simon, in, the, final, episode, of, series, 1, ,, after, he, slept, with, Jenny, ,, and, is, not,
		seen, again, ., Phoebe, Thomas, played, Cheryl, Cassidy, ,, Pauline, 's, friend, and, also, a, year,
		11, pupil, in, Simon, 's, class, ., Dumped, her, boyfriend, following, Simon, 's, advice, after, he,
		would, n't, have, sex, with, her, but, later, realised, this, was, due, to, him, catching, crabs,
		off, her, friend, Pauline, .]
 		and TokenIndexers : {'tokens': 'PretrainedTransformerMismatchedIndexer'} 
 	 spans: ListField of 2145 SpanFields : 
 	 SpanField with spans: (0, 0). 
 	 SpanField with spans: (0, 1). 
 	 SpanField with spans: (0, 2). 
 	 SpanField with spans: (0, 3). 
 	 SpanField with spans: (0, 4). 
 	 SpanField with spans: (0, 5). 
 	 SpanField with spans: (0, 6). 
 	 SpanF

In [11]:

torch.cuda.empty_cache()

In [10]:
from torch.optim import AdamW, SGD, Adam

# Define the AdamW optimizer
optimizer = Adam(model.parameters(), lr=0.001) 
# Create the trainer and train 
torch.cuda.empty_cache()
trainer = GradientDescentTrainer(
    model=model,
    optimizer=optimizer,
    data_loader=train_data_loader,
    validation_data_loader=validation_data_loader,
    num_epochs=3,
    cuda_device=0,  # set to -1 to use CPU 
    use_amp = False,	# Set to True to use automatic mixed precision
    num_gradient_accumulation_steps =3,
    grad_norm = 1.0,
    grad_clipping = 1.0,
)

trainer.train()

You provided a validation dataset but patience was set to None, meaning that early stopping is disabled
f:\UvA 2023\DL 4 NLP\Project\DLNLP-pronoun-resolution\DLNLP-pronoun-resolution\spanbert_env\lib\site-packages\torch\cuda\memory.py:329: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(


  0%|          | 0/223 [00:00<?, ?it/s]

f:\UvA 2023\DL 4 NLP\Project\DLNLP-pronoun-resolution\DLNLP-pronoun-resolution\spanbert_env\lib\site-packages\allennlp\nn\util.py:1553: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\tensor\python_tensor.cpp:85.)
  return torch.cuda.LongTensor(size, device=device).fill_(1).cumsum(0) - 1


KeyboardInterrupt: 